TIF images → `Extract-Channels` → `Visualize-Inputs` → `Predict-Segments` → `Visualize-Segmentation` → predicted segmentations

From source image, extract the nuclear & membrane channels. Save each to npz.

Not sure how we'll configure channel extraction. Maybe should do that individually.

In [1]:
import os

In [2]:
# Parameters (overridden by notebook parameters)
input_path = os.path.expanduser('~') + '/Downloads/Xenium_FFPE_Human_Breast_Cancer_Rep1_if_image.tif'
output_path = './saves'

In [3]:
import tifffile
import numpy as np

tif = tifffile.TiffFile(input_path)

# The page numbers need to be parameterized somehow…
# 2 lists of page numbers?? what if multiple input files? (ark-example data set)

# An array of shape [num_inputs, height, width] containing intensity of membrane channel
input_membrane = (tif.pages[0].asarray() + tif.pages[1].asarray())[np.newaxis, ...]

# An array of shape [num_inputs, height, width] containing intensity of nuclear channel
input_nuclear = tif.pages[2].asarray()[np.newaxis, ...]

tif.close()

In [4]:
np.savez_compressed(output_path + '/input_channels.npz', nuclear=input_nuclear, membrane=input_membrane)